For DYNAMIC Peak count and depolarization start times

version 6b changelist: 
- Per ROI instead of global median ROI
- Added troubleshooting functionalities (grey in plot is detected but filtered out of data table)
#
TO DO:
- Add summary page with means and standard deviation for each ROI on 1st sheet, keeping the other two sheets
- Possible correction for stimulation rate?
- Flagging? i.e. 1Hz cannot have APD90 of >1000ms, etc
- Alternans flagging (duration alternans)
- DAD and EAD flagging
- Fix occasional drifts
- Fix low amplitude peak detection, occasional
- Maybe start_idx (depolarization start) needs better detection settings?
QUESTIONS?: huynh.trung@mayo.edu


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
from itertools import groupby
from operator import itemgetter

# === Config ===
excel_path = r"C:\Users\m254292\Downloads\Wei Book5.xlsx"
band_factor     = 1.5
upstroke_min    = 0.06

APPLY_FILTERS        = True    # toggle filters
GREY_OUT_UNFILTERED  = True    # grey removed events on plots

xls = pd.ExcelFile(excel_path)

def analyze_roi_signal(time, signal, sample_name, roi_label,
                       apply_filters=True,
                       grey_out_unfiltered=True):
    # --- Prep ---
    signal = pd.to_numeric(signal, errors='coerce').dropna().values
    time   = time[: len(signal)]

    Fmin       = np.min(signal)
    normalized = (signal - Fmin) / Fmin
    smoothed   = gaussian_filter1d(normalized, sigma=1)

    slope_initial = np.gradient(smoothed, time)

    # Photobleaching correction
    flat_thresh = np.percentile(np.abs(slope_initial), 20)
    flat_idxs   = np.where(np.abs(slope_initial) < flat_thresh)[0]
    if flat_idxs.size >= 2:
        bleach_curve = np.interp(time, time[flat_idxs], smoothed[flat_idxs])
        bleach_curve[bleach_curve == 0] = 1
        smoothed = smoothed / bleach_curve

    # Thresholds
    s_max, s_med = np.max(smoothed), np.median(smoothed)
    dr            = s_max - s_med
    rising_thresh = 0.05 * dr
    depol_thresh  = 0.02 * dr

    slope = np.gradient(smoothed, time)

    # Rising-edge groups
    edges  = np.where(slope > rising_thresh)[0]
    groups = []
    for k, g in groupby(enumerate(edges), lambda x: x[0] - x[1]):
        grp = [i for _, i in g]
        if len(grp) >= 5:
            groups.append(grp)

    # Candidate peaks
    peeks = []
    for grp in groups:
        start = grp[0]
        end   = min(grp[-1] + 20, len(smoothed))
        peeks.append(np.argmax(smoothed[start:end]) + start)

    # De-dupe
    filt_peeks = []
    for p in peeks:
        if not filt_peeks or p - filt_peeks[-1] > 40:
            filt_peeks.append(p)

    # Metrics
    rows = []
    for pk in filt_peeks:
        window = range(max(0, pk - 100), pk)
        cand   = [i for i in window if slope[i] < depol_thresh]
        if not cand:
            continue
        start_idx = cand[-1]

        pre       = smoothed[max(0, start_idx - 50): start_idx]
        baseline  = np.min(pre)
        peak_val  = smoothed[pk]
        amp       = peak_val - baseline

        # APD90
        lvl90 = baseline + 0.1 * amp
        repol90_idx = next((i for i in range(pk + 1, len(smoothed)) if smoothed[i] <= lvl90), None)
        if repol90_idx is None:
            continue
        repol90_time  = time[repol90_idx]
        repol90_level = smoothed[repol90_idx]
        apd90         = repol90_time - time[start_idx]

        # APD50
        lvl50 = baseline + 0.5 * amp
        repol50_idx = next((i for i in range(pk + 1, len(smoothed)) if smoothed[i] <= lvl50), None)
        if repol50_idx is None:
            continue
        apd50 = time[repol50_idx] - time[start_idx]

        ratio50_90 = apd50 / apd90 if apd90 > 0 else np.nan
        upstroke   = time[pk] - time[start_idx]

        rows.append({
            'Depolarization_Start_Time_s': time[start_idx],
            'Peak_Time_s'                : time[pk],
            'Amplitude'                  : amp,
            'APD50_s'                    : apd50,
            'APD90_s'                    : apd90,
            'APD50_90_Ratio'             : ratio50_90,
            'Upstroke_Time_s'            : upstroke,
            'Repolarization_Time_s'      : repol90_time,
            'Repolarization_Level'       : repol90_level
        })

    # DataFrame
    df_res_raw = pd.DataFrame(rows).reset_index().rename(columns={'index': '_orig'})
    df_res     = df_res_raw.copy()

    # --- Filters ---
    if apply_filters and not df_res.empty:
        # Upstroke filter
        df_res = df_res[df_res['Upstroke_Time_s'] > upstroke_min]

        # Symmetric APD90 trim
        median_apd90 = df_res['APD90_s'].median()
        lower = median_apd90 / band_factor
        upper = median_apd90 * band_factor
        df_res = df_res[(df_res['APD90_s'] >= lower) & (df_res['APD90_s'] <= upper)]

        df_res = df_res.reset_index(drop=True)

    # Figure out which events were removed (only for plotting, not for summary printing)
    removed = (df_res_raw[~df_res_raw['_orig'].isin(df_res['_orig'])] 
               if apply_filters and grey_out_unfiltered else
               pd.DataFrame(columns=df_res_raw.columns))

    expected = [
        'Depolarization_Start_Time_s', 'Peak_Time_s', 'Amplitude',
        'APD50_s', 'APD90_s', 'APD50_90_Ratio', 'Upstroke_Time_s',
        'Repolarization_Time_s', 'Repolarization_Level'
    ]
    df_res = df_res.reindex(columns=['_orig'] + expected)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(time, smoothed, label='Signal')

    # Grey-out removed (if desired)
    if grey_out_unfiltered and not removed.empty:
        for i, t in enumerate(removed['Peak_Time_s']):
            ax.axvline(t, color='grey', linestyle='--', alpha=0.3,
                       label='Removed Peak' if i == 0 else None)
        for i, t in enumerate(removed['Depolarization_Start_Time_s']):
            ax.axvline(t, color='grey', linestyle=':', alpha=0.3,
                       label='Removed Depol Start' if i == 0 else None)
        for i, row in enumerate(removed.itertuples()):
            ax.hlines(y=row.Repolarization_Level,
                      xmin=row.Depolarization_Start_Time_s,
                      xmax=row.Repolarization_Time_s,
                      color='grey', linestyle='-', linewidth=2, alpha=0.3,
                      label='Removed APD90' if i == 0 else None)

    # Kept events
    for i, t in enumerate(df_res['Peak_Time_s']):
        ax.axvline(t, color='green', linestyle='--', alpha=0.8,
                   label='Peak' if i == 0 else None)
    for i, t in enumerate(df_res['Depolarization_Start_Time_s']):
        ax.axvline(t, color='black', linestyle='--', alpha=0.5,
                   label='Depol Start' if i == 0 else None)
    for i, row in enumerate(df_res.itertuples()):
        ax.hlines(y=row.Repolarization_Level,
                  xmin=row.Depolarization_Start_Time_s,
                  xmax=row.Repolarization_Time_s,
                  color='red', linestyle='-', linewidth=2, alpha=0.8,
                  label='APD90' if i == 0 else None)

    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("ΔF/Fmin")
    ax.set_title(f"{sample_name} | {roi_label} (bleach-corrected)")
    ax.grid(True)
    fig.tight_layout()

    return sample_name, roi_label, df_res, fig   # <- only filtered df returned


# === Main ===
all_results = []
plots       = []

for sheet_name in xls.sheet_names:
    raw = xls.parse(sheet_name, header=None, nrows=1)
    if raw.dropna(how='all').empty:
        continue
    sample_name = str(raw.iloc[0, 0])
    if sample_name.lower().endswith('.nd2'):
        sample_name = sample_name[:-4]

    df = xls.parse(sheet_name, header=1)
    if df.dropna(how='all').empty:
        continue
    df.columns = df.columns.astype(str).str.strip()

    time_cols = [c for c in df.columns if 'time' in c.lower()]
    if not time_cols:
        continue
    time = pd.to_numeric(df[time_cols[0]], errors='coerce').values

    roi_cols = [c for c in df.columns if c != time_cols[0] and 'Mono' in c]
    for roi in roi_cols:
        samp, roi_label, res_df, fig = analyze_roi_signal(
            time, df[roi], sample_name, roi,
            apply_filters=APPLY_FILTERS,
            grey_out_unfiltered=GREY_OUT_UNFILTERED
        )

        res_df.insert(0, 'Sample', samp)
        res_df.insert(1, 'ROI', roi_label)

        all_results.append(res_df)
        plots.append((samp, roi_label, fig))

# === Only filtered summary ===
if all_results:
    summary_df = pd.concat(all_results, ignore_index=True)
    print("Filtered summary:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(summary_df)


C:\Users\m254292\AppData\Local\Temp\ipykernel_20536\4222943473.py:144: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(12, 5))


Filtered summary:


,Sample,ROI,_orig,Depolarization_Start_Time_s,Peak_Time_s,Amplitude,APD50_s,APD90_s,APD50_90_Ratio,Upstroke_Time_s,Repolarization_Time_s,Repolarization_Level
0,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),0,0.920,1.060,0.171627,0.520,0.600,0.866667,0.140,1.520,0.017536
1,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),1,1.921,2.061,0.171843,0.520,0.600,0.866667,0.140,2.521,0.017291
2,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),2,2.921,3.061,0.171740,0.520,0.600,0.866667,0.140,3.521,0.018755
3,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),3,3.921,4.061,0.167362,0.520,0.600,0.866667,0.140,4.521,0.016646
4,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),4,4.921,5.061,0.173397,0.521,0.601,0.866889,0.140,5.522,0.018248
5,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),5,5.922,6.062,0.168685,0.520,0.600,0.866667,0.140,6.522,0.017227
6,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),6,6.922,7.062,0.167723,0.520,0.600,0.866667,0.140,7.522,0.018625
7,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),7,7.902,8.062,0.169711,0.540,0.620,0.870968,0.160,8.522,0.017210
8,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),8,8.922,9.062,0.166483,0.521,0.601,0.866889,0.140,9.523,0.018238
9,63C42-DF_6-21-D38-No treatment-0001,#1 (Mono - GFP),9,9.903,10.063,0.167064,0.540,0.620,0.870968,0.160,10.523,0.018147


In [ ]:
import os
import io

# === After you’ve built `all_results` and `summary_df` ===
# (and assuming you modified analyze_roi_signal to return the Figure object:)
#
# e.g. def analyze_roi_signal(...):
#       …
#       fig, ax = plt.subplots(figsize=(12,5))
#       ax.plot(...)
#       … 
#       return sample_name, roi_label, df_res, fig
#
# And in your loop you collected:
#    plots = []  # list of (sample, roi, fig)
#    for …:
#        samp, roi, df_res, fig = analyze_roi_signal(...)
#        plots.append((samp, roi, fig))
#        # insert sample/ROI into df_res as before
#        all_results.append(df_res)
#
# At this point you have:
#    summary_df  ← pd.concat(all_results)
#    plots       ← list of (sample, roi, fig)

# === Dynamically build out_path using os.path (Option 1) ===
folder, basename = os.path.split(excel_path)
name, ext       = os.path.splitext(basename)
new_name        = f"{name} batch analysis{ext}"
out_path        = os.path.join(folder, new_name)

# === Write summary_df and plots into the new workbook ===
with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
    # 1) summary table
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

    # 2) embed plots
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Plots')

    img_row = 0
    for sample, roi, fig in plots:
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)

        worksheet.write(img_row, 0, f"{sample} | {roi}")
        img_row += 1

        worksheet.insert_image(
            img_row, 0,
            f"{sample}_{roi}.png",
            {'image_data': buf, 'x_scale': 0.8, 'y_scale': 0.8}
        )
        img_row += 30

print(f"✅ Saved batch analysis to:\n  {out_path}")

✅ Saved batch analysis to:
  C:\Users\m254292\Downloads\Wei Book2 batch analysis.xlsx


To Do: 
- Add APD50
- Add ratio of APD50/90
